In [17]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch.nn.functional as F
import numpy as np
import itertools

def flatten(l):
    return list(itertools.chain.from_iterable(l))

seqs = ['ghatmasala','nicela','c-pakodas']

# make <pad> idx 0
vocab = ['<pad>'] + sorted(list(set(flatten(seqs))))

# make model
embed = nn.Embedding(len(vocab), 10)
lstm = nn.LSTM(10, 5)

vectorized_seqs = [[vocab.index(tok) for tok in seq] for seq in seqs]

# get the length of each seq in your batch
seq_lengths = torch.LongTensor(list(map(len, vectorized_seqs)))

# dump padding everywhere, and place seqs on the left.
# NOTE: you only need a tensor as big as your longest sequence
seq_tensor = Variable(torch.zeros((len(vectorized_seqs), seq_lengths.max()))).long()
for idx, (seq, seqlen) in enumerate(zip(vectorized_seqs, seq_lengths)):
    seq_tensor[idx, :seqlen] = torch.LongTensor(seq)


# SORT YOUR TENSORS BY LENGTH!
seq_lengths, perm_idx = seq_lengths.sort(0, descending=True)
seq_tensor = seq_tensor[perm_idx]

# utils.rnn lets you give (B,L,D) tensors where B is the batch size, L is the maxlength, if you use batch_first=True
# Otherwise, give (L,B,D) tensors
seq_tensor = seq_tensor.transpose(0,1) # (B,L,D) -> (L,B,D)
print(seq_tensor)
# embed your sequences
seq_tensor = embed(seq_tensor)

# pack them up nicely
packed_input = pack_padded_sequence(seq_tensor, seq_lengths.cpu().numpy())

# throw them through your LSTM (remember to give batch_first=True here if you packed with it)
packed_output, (ht, ct) = lstm(packed_input)

# unpack your output if required
output, _ = pad_packed_sequence(packed_output)
print(output)

# Or if you just want the final hidden state?

Variable containing:
    6     3    12
    7     1     8
    2    14     3
   16     2     5
   11     9    10
    2    13     2
   15     4     0
    2     2     0
   10    15     0
    2     0     0
[torch.LongTensor of size 10x3]

Variable containing:
(0 ,.,.) = 
 -0.0902  0.0142  0.0054 -0.1126  0.0394
 -0.0251  0.0552  0.0039  0.0441 -0.0241
 -0.0811 -0.1490 -0.1966  0.1724  0.1074

(1 ,.,.) = 
  0.0223  0.0563  0.1778 -0.1500  0.0378
  0.2266 -0.1383  0.1943 -0.1969 -0.1424
 -0.0822  0.0351  0.0002  0.0687  0.0513

(2 ,.,.) = 
 -0.0147  0.1148 -0.1016 -0.2115  0.1299
  0.1662  0.0198 -0.0919 -0.0830 -0.0050
 -0.1020  0.0612  0.0117  0.1067  0.0106

(3 ,.,.) = 
 -0.0399  0.0958  0.1701 -0.0751 -0.1548
  0.0436  0.0814 -0.3451 -0.2643  0.0989
 -0.0820  0.0803 -0.2821 -0.1690  0.0578

(4 ,.,.) = 
  0.0523 -0.1280  0.3578 -0.3141 -0.1152
  0.0697  0.1416 -0.1687 -0.2225  0.0639
 -0.2033  0.0992 -0.3016 -0.0572  0.1865

(5 ,.,.) = 
 -0.0141  0.0551 -0.0620 -0.2223  0.0322
 -0.0216  0.

In [4]:
packed_input

PackedSequence(data=Variable containing:
 0.2458 -1.2492  1.7491  0.3977  0.8368 -2.9893  1.0737 -0.6264 -0.2431 -0.3737
-0.9855 -0.7507  0.6510  0.8058 -0.7101  0.1943 -0.3969 -0.3953  1.6264  1.6685
 1.1175 -1.7670  0.9485  1.0839  0.2394  0.0025 -1.3496 -0.7555 -0.4157 -0.1383
 1.1401  1.2943 -0.1035  0.5639  0.8211 -1.2519 -1.8266  1.3199  1.3147  0.8338
-0.3011  0.3611  0.5286 -0.9234  0.5133 -0.6587  0.8846 -0.4795  0.0756 -0.1951
 0.7663 -0.7539 -1.0057  0.1264  0.1875  0.8970 -0.4165 -1.0588  0.0584  0.1958
 1.1806  0.2788 -0.1120 -0.7458  0.6243  0.3237  0.7052  0.7941 -1.0058  1.4893
-0.4987 -0.7722  0.8738  1.2237  0.2091 -1.1581 -1.6838 -1.4353  0.5427 -1.0765
-0.9855 -0.7507  0.6510  0.8058 -0.7101  0.1943 -0.3969 -0.3953  1.6264  1.6685
 1.3021  0.2287 -1.2955  0.5958 -0.5541  0.9566  0.3134  1.2365 -0.5294  1.2598
 1.1806  0.2788 -0.1120 -0.7458  0.6243  0.3237  0.7052  0.7941 -1.0058  1.4893
 0.3945  1.1748 -0.7199 -0.3348  1.3338 -0.6700  0.5455 -0.8923  0.3145  1.5411

In [16]:
seq_tensor[2, :, :]

Variable containing:
 1.1806  0.2788 -0.1120 -0.7458  0.6243  0.3237  0.7052  0.7941 -1.0058  1.4893
-0.4987 -0.7722  0.8738  1.2237  0.2091 -1.1581 -1.6838 -1.4353  0.5427 -1.0765
-0.9855 -0.7507  0.6510  0.8058 -0.7101  0.1943 -0.3969 -0.3953  1.6264  1.6685
[torch.FloatTensor of size 3x10]

In [5]:
seq_tensor, seq_lengths.cpu().numpy()

(Variable containing:
 (0 ,.,.) = 
 
 Columns 0 to 8 
    0.2458 -1.2492  1.7491  0.3977  0.8368 -2.9893  1.0737 -0.6264 -0.2431
  -0.9855 -0.7507  0.6510  0.8058 -0.7101  0.1943 -0.3969 -0.3953  1.6264
   1.1175 -1.7670  0.9485  1.0839  0.2394  0.0025 -1.3496 -0.7555 -0.4157
 
 Columns 9 to 9 
   -0.3737
   1.6685
  -0.1383
 
 (1 ,.,.) = 
 
 Columns 0 to 8 
    1.1401  1.2943 -0.1035  0.5639  0.8211 -1.2519 -1.8266  1.3199  1.3147
  -0.3011  0.3611  0.5286 -0.9234  0.5133 -0.6587  0.8846 -0.4795  0.0756
   0.7663 -0.7539 -1.0057  0.1264  0.1875  0.8970 -0.4165 -1.0588  0.0584
 
 Columns 9 to 9 
    0.8338
  -0.1951
   0.1958
 
 (2 ,.,.) = 
 
 Columns 0 to 8 
    1.1806  0.2788 -0.1120 -0.7458  0.6243  0.3237  0.7052  0.7941 -1.0058
  -0.4987 -0.7722  0.8738  1.2237  0.2091 -1.1581 -1.6838 -1.4353  0.5427
  -0.9855 -0.7507  0.6510  0.8058 -0.7101  0.1943 -0.3969 -0.3953  1.6264
 
 Columns 9 to 9 
    1.4893
  -1.0765
   1.6685
 
 (3 ,.,.) = 
 
 Columns 0 to 8 
    1.3021  0.2287 -1.295

In [6]:
seq_tensor.shape, seq_lengths

(torch.Size([10, 3, 10]), 
  10
   9
   6
 [torch.LongTensor of size 3])